In [1]:
from selenium.webdriver.chrome.options import Options
from os import path, listdir, rename, getcwd
from fake_useragent import UserAgent
from dateutil.parser import parse
from selenium import webdriver
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import pickle
import time
import re

In [2]:
def chrome_user_agent():
    return UserAgent().chrome

chrome_user_agent()

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36'

In [3]:
def chrome_driver():
    options = Options()
    # options.add_argument('--headless')
    options.add_argument("window-size=1600,1080")
    options.add_argument(f'user-agent={chrome_user_agent()}')
        
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")

    return webdriver.Chrome(options=options)

In [4]:
def check_products(soup, category, products_found = []):
    print(category)
    print('--------------------------------------')

    sku_list = soup.find('ol', {'class', 'sku-item-list'}).findAll('li', {'class', 'sku-item'})

    for sku in sku_list:
        title = sku.find('div', {'class', 'sku-title'}).find('a')

        print(title.text.strip()[:40])

        if re.search("combo", title['href']):
            if sku.find('div', {'class', 'combo-add-to-cart-button'}).find('svg'):
                name = title.text.strip()[:40]
                price = sku.find('div', {'class', 'priceView-customer-price'}).find('span').text[1:]
                button = sku.find('button', {'class', 'add-to-cart-button'}).text.strip()
                print(category, name, price, datetime.now())
        else:
            if sku.find('button', {'class', 'add-to-cart-button'}).find('svg'):
                name = title.text.strip()[:40]
                price = sku.find('div', {'class', 'priceView-customer-price'}).find('span').text[1:]
                button = sku.find('div', {'class', 'sku-list-item-button'}).find('button').text.strip()
                print(category, name, price, datetime.now())

    if soup.find('div', {'class', 'footer-category'}).find_next_sibling('div') and not soup.find('a', {'class', 'sku-list-page-next disabled'}):
        driver.get(soup.find('a', {'class', 'sku-list-page-next'})['href'])
        time.sleep(5)
        check_products(BeautifulSoup(driver.page_source, 'html.parser'), category, products_found)
    else:
        return products_found

In [5]:
def get_soup(url):
    driver.get(url)
    time.sleep(5)
    return BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
def check_nintendo_switches():
    nintendo_url = 'https://www.bestbuy.com/site/nintendo-switch/nintendo-switch-consoles/pcmcat1484077694025.c?id=pcmcat1484077694025'
    return check_products(get_soup(nintendo_url), 'Switch')

In [7]:
def check_xbox_consoles():
    xbox_url = 'https://www.bestbuy.com/site/xbox-series-x-and-s/xbox-series-x-and-s-consoles/pcmcat1586900952752.c?id=pcmcat1586900952752'
    return check_products(get_soup(xbox_url), "Xbox")

In [8]:
def check_ps5_consoles():
    ps5_url = 'https://www.bestbuy.com/site/playstation-5/ps5-consoles/pcmcat1587395025973.c?id=pcmcat1587395025973'
    return check_products(get_soup(ps5_url), "PS5")

In [9]:
def check_rtx_3000_series():
    rtx_url = 'https://www.bestbuy.com/site/computer-cards-components/video-graphics-cards/abcat0507002.c?id=abcat0507002&qp=gpusv_facet%3DGraphics%20Processing%20Unit%20(GPU)~NVIDIA%20GeForce%20RTX%20'

    rtx_list = ['3060', '3060 Ti', '3070', '3080', '3090']

    for gpu in rtx_list:
        return check_products(get_soup(rtx_url+gpu.replace(' ','%20')), gpu)

In [10]:
def check_radeon_rx_series():
    radeon_url = 'https://www.bestbuy.com/site/computer-cards-components/video-graphics-cards/abcat0507002.c?id=abcat0507002&qp=gpusv_facet%3DGraphics%20Processing%20Unit%20(GPU)~AMD%20Radeon%20RX%20'

    radeon_list = ['6700 XT', '6800 XT', '6800', '6900 XT']

    for gpu in radeon_list:
        return check_products(get_soup(radeon_url+gpu.replace(' ','%20')), gpu)

In [11]:
driver = chrome_driver()

In [12]:
# print(check_nintendo_switches())

In [13]:
print(check_xbox_consoles())

Xbox
--------------------------------------
Package - Microsoft - Xbox Series S 512 
Microsoft - Xbox Series S 512 GB All-Dig
Microsoft - Xbox Series X 1TB Console - 
Package - Microsoft - Xbox Series S 512 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series S 512 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series S 512 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series S 512 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
Package - Microsoft - Xbox Series X 1TB 
[]


In [14]:
# print(check_ps5_consoles())

In [15]:
# print(check_rtx_3000_series())

In [16]:
# print(check_radeon_rx_series())

In [17]:
# driver.close()